In [ ]:
import utils as utils
import network as nw
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn as nn
import torch
from tqdm import tqdm
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import numpy as np
import train as trainer
import importlib as im

In [ ]:
DATASET_DIR = 'D:/dataset_car/kcar_preprocessed/kcar'
BATCH_SIZE = 16
LEARNING_RATE = 0.001
EPOCH = 10
SAVE_DIR = './result/'

In [ ]:
# GET DEVICE AND CHECK.
gpu_flag = torch.cuda.is_available()
device = torch.device("cuda" if gpu_flag else "cpu")
print("GPU INFO : ", torch.cuda.get_device_name(device))

In [ ]:
# LOAD DATASET
d_total = utils.get_dataset(DATASET_DIR)
_, d_total = utils.split_dataset(d_total, (4,1))
# _, d_total = utils.split_dataset(d_total, (99, 1))
print("DATA LOADING DONE.")

In [ ]:
# cross validation for epoch
im.reload(trainer)

EPOCH = 10
BATCH_SIZE = 16
LEARNING_RATE = 0.001

cv_result = trainer.cross_validation(
    dataset=d_total,
    device=device,
    max_epoch=EPOCH,
    batch_size=BATCH_SIZE,
    lr=LEARNING_RATE,
    n_split=2,
    modelType='m2'
    )

In [ ]:
x_axis = [i for i in range(len(cv_result))]
y_axis = list(cv_result)
plt.plot(x_axis, y_axis)

In [ ]:
## Training
im.reload(trainer)
EPOCH = 1
BATCH_SIZE=16
LEARNING_RATE=0.001
SAVE_DIR = './result/resNet18'

last_model, train_loss_arr, test_loss_arr, test_acc_arr = trainer.train(
    dataset=d_total,
    device=device,
    max_epoch=EPOCH,
    batch_size=BATCH_SIZE,
    lr=LEARNING_RATE,
    modelType='m1',
    save_dir=SAVE_DIR
)

# After train, see structure
print(last_model)

In [ ]:
# training and test loss by epoch.

x_axis = [i for i in range(EPOCH)]
plt.plot(x_axis, train_loss_arr)
plt.plot(x_axis, test_loss_arr)
plt.show()

In [ ]:
# obtain first 2 convolutional layer filter maps.

filter_map = []
for param in list(last_model.parameters()):
    if param.dim() == 4:
        print(param.shape)
        filter_map.append(param)
    
last_model.cpu()
utils.visualize_filter(filter_map[0])
utils.visualize_filter(filter_map[1])

In [ ]:
# test accuracy by epoch, until epoch 30

x_axis = [i for i in range(30)]
test_acc_arr = [i.cpu() for i in test_acc_arr]
plt.plot(x_axis, test_acc_arr[:30])
plt.show()